# Phase 2: Portfolio Backtesting

Explore multi-asset portfolio strategies with different allocations:
- **Equal-weight**: Split capital evenly across all assets
- **Custom allocation**: 60/30/10 split (stocks/bonds/gold)
- **Rebalancing**: Compare monthly vs quarterly rebalancing
- **Factor-weighted**: Use Momentum to dynamically adjust weights

We use a simple 3-asset universe: SPY (stocks), BND (bonds), GLD (gold).


In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import pandas as pd

from qlib.backtesting import PortfolioBacktester
from qlib.data import DataLoader
from qlib.factors import Momentum
from qlib.metrics import max_drawdown, sharpe, sortino

plt.style.use("seaborn-v0_8")

In [ ]:
symbols = ["SPY", "BND", "GLD"]
start = "2015-01-01"

print(f"Loading universe: {symbols} starting {start}...")
universe = DataLoader.load_universe(symbols, start=start)
universe.head()

In [ ]:
# Extract close prices for plotting
close_prices = universe.xs("close", axis=1, level=1)

# Normalize to $1 start for comparison
normalized = close_prices / close_prices.iloc[0]

ax = normalized.plot(figsize=(10, 4), title="Normalized Asset Prices")
ax.set_ylabel("Growth of $1")
plt.show()

## Equal-Weight Portfolio

The simplest allocation: split capital evenly across all three assets (33% each). Rebalance monthly to maintain the allocation.

In [ ]:
bt_equal = PortfolioBacktester(universe, weights=None, rebalance_freq="ME")
rets_equal = bt_equal.run().dropna()

metrics_equal = {
    "sharpe": sharpe(rets_equal),
    "sortino": sortino(rets_equal),
    "max_drawdown": max_drawdown(rets_equal),
}
metrics_equal

In [ ]:
cum_equal = (1 + rets_equal).cumprod()
ax = cum_equal.plot(figsize=(10, 4), title="Equal-Weight Portfolio")
ax.set_ylabel("Growth of $1")
plt.show()

## Custom Allocation: 60/30/10

A classic allocation tilted toward stocks:
- 60% SPY (stocks)
- 30% BND (bonds)
- 10% GLD (gold)

In [ ]:
custom_weights = {"SPY": 0.6, "BND": 0.3, "GLD": 0.1}
bt_custom = PortfolioBacktester(universe, weights=custom_weights, rebalance_freq="ME")
rets_custom = bt_custom.run().dropna()

metrics_custom = {
    "sharpe": sharpe(rets_custom),
    "sortino": sortino(rets_custom),
    "max_drawdown": max_drawdown(rets_custom),
}
metrics_custom

In [ ]:
cum_custom = (1 + rets_custom).cumprod()
ax = cum_custom.plot(figsize=(10, 4), title="60/30/10 Portfolio")
ax.set_ylabel("Growth of $1")
plt.show()

## Rebalancing Frequency

Compare monthly vs quarterly rebalancing on the 60/30/10 portfolio. More frequent rebalancing maintains target weights but incurs more trading costs.

In [ ]:
bt_monthly = PortfolioBacktester(universe, weights=custom_weights, rebalance_freq="ME")
bt_quarterly = PortfolioBacktester(universe, weights=custom_weights, rebalance_freq="QE")

rets_monthly = bt_monthly.run().dropna()
rets_quarterly = bt_quarterly.run().dropna()

metrics_monthly = {
    "sharpe": sharpe(rets_monthly),
    "sortino": sortino(rets_monthly),
    "max_drawdown": max_drawdown(rets_monthly),
}
metrics_quarterly = {
    "sharpe": sharpe(rets_quarterly),
    "sortino": sortino(rets_quarterly),
    "max_drawdown": max_drawdown(rets_quarterly),
}

print("Monthly rebalancing:", metrics_monthly)
print("Quarterly rebalancing:", metrics_quarterly)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
(1 + rets_monthly).cumprod().plot(ax=ax, label="Monthly")
(1 + rets_quarterly).cumprod().plot(ax=ax, label="Quarterly")
ax.set_title("Rebalancing Frequency Comparison (60/30/10)")
ax.set_ylabel("Growth of $1")
ax.legend()
plt.show()

## Factor-Weighted Portfolio

Instead of fixed weights, use Momentum to dynamically allocate. Assets with positive momentum get higher weights, negative momentum get lower or zero weights.

In [ ]:
# Compute momentum for each asset
lookback = 20
factor = Momentum(lookback=lookback)

momentum_signals = pd.DataFrame(index=universe.index)
for symbol in symbols:
    asset_data = universe[symbol]
    momentum_signals[symbol] = factor.compute(asset_data)

# Convert momentum to positive weights (clip negative to zero)
# This creates a "long-only momentum" strategy
positive_momentum = momentum_signals.clip(lower=0)

momentum_signals.tail()

In [ ]:
bt_momentum = PortfolioBacktester(universe)
rets_momentum = bt_momentum.run(signals=positive_momentum).dropna()

metrics_momentum = {
    "sharpe": sharpe(rets_momentum),
    "sortino": sortino(rets_momentum),
    "max_drawdown": max_drawdown(rets_momentum),
}
metrics_momentum

In [ ]:
cum_momentum = (1 + rets_momentum).cumprod()
ax = cum_momentum.plot(figsize=(10, 4), title="Momentum-Weighted Portfolio")
ax.set_ylabel("Growth of $1")
plt.show()

## Strategy Comparison

Compare all strategies side by side.

In [ ]:
comparison = pd.DataFrame(
    {
        "Equal-Weight": metrics_equal,
        "60/30/10": metrics_custom,
        "Monthly Rebal": metrics_monthly,
        "Quarterly Rebal": metrics_quarterly,
        "Momentum": metrics_momentum,
    }
).T

comparison.round(3)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

(1 + rets_equal).cumprod().plot(ax=ax, label="Equal-Weight")
(1 + rets_custom).cumprod().plot(ax=ax, label="60/30/10")
(1 + rets_momentum).cumprod().plot(ax=ax, label="Momentum")

ax.set_title("Cumulative Returns: Portfolio Comparison")
ax.set_ylabel("Growth of $1")
ax.legend()
plt.show()